In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2 as cv2


> # **1. processdata**

In [ ]:
data_train = []

In [ ]:
from IPython.core.display import Image
import cv2
import os
import random
import shutil 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np
count = 0;
from google.colab.patches import cv2_imshow
def draw_bounding_box(image, annotation_text_file):
  with open(annotation_text_file, 'r') as file:
    annotation = file.read().split('\n')[:-1]
    if annotation != []:
      # with open(annotation_text_file, 'r') as file:
      #   annotation = file.read().split('\n')[:-1]
      annotation = [ x.split(" ") for x in annotation]
      annotation = [ list(map(float,x)) for x in annotation]

        
      annotations = np.array(annotation) #has a from [[class_id, x_center, y_center, bbox_width, bbox_height]]

      print(f"annotation value before de-normalizaiton: {annotations}")
      h , w, _ = image.shape
      print("width:",h)
      print("height:",w)
      # de-normalize annotations
      annotations_cp = np.copy(annotations)
      annotations_cp[:, [1,3]] = annotations[:, [1,3]] * w # x_center, bbox_width
      annotations_cp[:, [2,4]] = annotations[:, [2,4]] * h # y_center, bbox_height
      print(f"annotation value after de-normolization: {annotations_cp}")

      #convert to (xmin, xmax, ymin, ymax) to draw cv2.rectangle
      annotations_cp[ :,1] = (annotations_cp[ :,1] - annotations_cp[ :,3]/2)
      annotations_cp[ :,2] = (annotations_cp[ :,2] - annotations_cp[ :,4]/2)
      annotations_cp[ :,3] = (annotations_cp[ :,1] + annotations_cp[ :,3])
      annotations_cp[ :,4] = (annotations_cp[ :,2] + annotations_cp[ :,4])

      for single_annotation in annotations_cp:
        obj_cls, x0, y0, x1, y1 = single_annotation
        start_point = (int(x0), int(y0))
        end_point = (int(x1), int(y1))
        color=(255,0,0)
        thickness=2
        cv2.rectangle(image, start_point, end_point, color, thickness)
        cv2_imshow(image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    

In [ ]:
# from glob import glob
# %cd /content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster
# annotation_file = glob("/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/images/lables/*.txt")
# for annotation in annotation_file:
#   file_name = annotation.split("/")[-1]
#   file = "/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/images/" + file_name.replace('txt','jpg')
#   print(file)
#   img = cv2.imread(file)
#   draw_bounding_box(img, annotation)

In [ ]:
__class_name_to_id_napping = {"nine-dash line":0}
#step 1: extract xml info
def extract_inf(xml_file):
  root = ET.parse(xml_file).getroot()
  info_dict = {}
  info_dict['bboxes'] = []

  for elem in root:
    if elem.tag == "filename":
      info_dict['filename'] = elem.text
      info_dict['filename'] = "/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/images/" + info_dict['filename']
    elif elem.tag == 'object':
      bbox = {}
      for subelem in elem:
        if subelem.tag == 'name':
          bbox["class"] = "yes"
        elif subelem.tag == 'bndbox':
          for subsubelem in subelem:
            bbox[subsubelem.tag] = int(subsubelem.text)
      row = [info_dict['filename'],  bbox['xmin'] , bbox['ymin'], bbox['xmax'], bbox['ymax'], bbox["class"]]
      df.append(row)

In [ ]:
import pandas as pd
import os
from glob import glob
import xml.etree.ElementTree as ET
count = 0;
df = []
val = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/images/annotations/*.xml')
for file in val:
    info_dict = extract_inf(file)
    count +=1;
print(count)
data_train = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])


494


In [ ]:
data_train = data_train.sample(frac = 1)
data_train.head

<bound method NDFrame.head of                                               filename  xmin  ymin  xmax  \
231  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    55    39   280   
23   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    76    89   162   
446  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   335    41   543   
399  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   513   592   839   
131  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    12    27   180   
..                                                 ...   ...   ...   ...   
143  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   107     7   236   
242  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   467   364  1458   
153  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   109    12   219   
29   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    33     9   181   
206  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    75    66   157   

     ymax dash  
231   340  yes  
23    200  yes  
446   

In [6]:
def check(img_name,df):
  for name in df['filename']:
    # if img_name == name.split("/")[-1] :
    if img_name == name:
      return True
  return False

In [ ]:
import os, sys
from glob import glob
import pandas as pd
import cv2 
import numpy as np

annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/images/lables/*.txt')

# df = []
count = 0;
count_train =0;
for file in annotations:
    filename = file.split('/')[-1]
    filenameJPG = filename.split('.')[0] + '.jpg'
    image_file = "/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/images/" + filenameJPG
    img = cv2.imread(image_file)
    height, width,_ = img.shape
    row = []
    with open("/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/images/lables/" + filename, 'r') as file:
        annotation = file.read().split('\n')[:-1]
        if annotation == [] :
            count +=1;
            continue; 
        
        annotation = [ x.split(" ") for x in annotation]
        annotation = [ list(map(float,x)) for x in annotation]
    # for annotation_s in annotation: 
    annotations = np.array(annotation) #has a from [[class_id, x_center, y_center, bbox_width, bbox_height]]
    # de-normalize annotations
    annotations_cp = np.copy(annotations)
    annotations_cp[:, [1,3]] = annotations[:, [1,3]] * width # x_center, bbox_width
    annotations_cp[:, [2,4]] = annotations[:, [2,4]] * height # y_center, bbox_height


    #convert to (xmin, xmax, ymin, ymax) to draw cv2.rectangle
    annotations_cp[ :,1] = (annotations_cp[ :,1] - annotations_cp[ :,3]/2)
    annotations_cp[ :,2] = (annotations_cp[ :,2] - annotations_cp[ :,4]/2)
    annotations_cp[ :,3] = (annotations_cp[ :,1] + annotations_cp[ :,3])
    annotations_cp[ :,4] = (annotations_cp[ :,2] + annotations_cp[ :,4])
 
    for single_annotation in annotations_cp:
        dash = "yes"
        obj_cls, x0, y0, x1, y1 = single_annotation
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
        row = [image_file,  x0 , y0, x1, y1, dash]
        df.append(row)
        count_train +=1;

data_train = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])

In [ ]:
print("img has value", count)
print("img has value", count_train)

data_train = data_train.sample(frac = 1)
data_train.head

img has value 0
img has value 1468


<bound method NDFrame.head of                                                filename  xmin  ymin  xmax  \
60    /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   288    25   462   
1461  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   377   163   520   
1768  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   272    32   673   
1104  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   479    98   759   
1262  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   346   112   648   
...                                                 ...   ...   ...   ...   
203   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   224   146   258   
423   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   257   178   280   
691   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   139   509   416   
197   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   192   797   590   
1434  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    34   158   257   

      ymax dash  
60     296  yes  
1461   35

read data background

In [ ]:
annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/images/*.jpg')

for file in annotations:
  if (check(file,data_train) == False):
    data = [file] + [0,0,0,0] + ['bg']
    df.append(data)

  data_train = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])

In [ ]:
print("img backgound", count)
data_train = data_train.sample(frac = 1)
data_train.head(20)

img backgound 0


,filename,xmin,ymin,xmax,ymax,dash
136,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,100,12,191,176,yes
1824,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,514,348,577,436,yes
234,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,209,183,251,245,yes
1277,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,916,390,1005,567,yes
437,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,22,42,186,248,yes
1944,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,267,0,618,402,yes
413,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,217,385,292,490,yes
1190,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,100,74,216,232,yes
1478,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,450,33,745,505,yes
1939,/content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...,405,348,522,498,yes


In [ ]:
data_train['dash'].value_counts()

yes    1971
bg        6
Name: dash, dtype: int64

save bbox and label


In [ ]:
data_train.to_csv('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/train/annotations.txt',index=False,header=False)

get annotation test data <br>
import

In [3]:
import os, sys
from glob import glob
import pandas as pd
import cv2 
import numpy as np


annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/images/lables/*.txt')

df = []
count = 0;
count_train =0;
for file in annotations:
    filename = file.split('/')[-1]
    filenameJPG = filename.split('.')[0] + '.jpg'
    image_file = "/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/images/" + filenameJPG
    img = cv2.imread(image_file)
    height, width,_ = img.shape
    row = []
    with open("/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/images/lables/" + filename, 'r') as file:
        annotation = file.read().split('\n')[:-1]
        if annotation == [] :
            count +=1;
            continue; 
        
        annotation = [ x.split(" ") for x in annotation]
        annotation = [ list(map(float,x)) for x in annotation]
    # for annotation_s in annotation: 
    annotations = np.array(annotation) #has a from [[class_id, x_center, y_center, bbox_width, bbox_height]]
    # de-normalize annotations
    annotations_cp = np.copy(annotations)
    annotations_cp[:, [1,3]] = annotations[:, [1,3]] * width # x_center, bbox_width
    annotations_cp[:, [2,4]] = annotations[:, [2,4]] * height # y_center, bbox_height


    #convert to (xmin, xmax, ymin, ymax) to draw cv2.rectangle
    annotations_cp[ :,1] = (annotations_cp[ :,1] - annotations_cp[ :,3]/2)
    annotations_cp[ :,2] = (annotations_cp[ :,2] - annotations_cp[ :,4]/2)
    annotations_cp[ :,3] = (annotations_cp[ :,1] + annotations_cp[ :,3])
    annotations_cp[ :,4] = (annotations_cp[ :,2] + annotations_cp[ :,4])
 
    for single_annotation in annotations_cp:
        dash = "yes"
        obj_cls, x0, y0, x1, y1 = single_annotation
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
        row = [image_file,  x0 , y0, x1, y1, dash]
        df.append(row)
        count_train +=1;

data_test = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])

In [4]:
print("img has value", count)
print("img has value", count_train)

data_test = data_test.sample(frac = 1)
data_test.head

img has value 0
img has value 244


<bound method NDFrame.head of                                               filename  xmin  ymin  xmax  \
229  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   293   295   372   
40   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   483   421   528   
85   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   177    46   440   
123  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   144    85   335   
150  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...    54    75   324   
..                                                 ...   ...   ...   ...   
64   /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   224   228   306   
116  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   353    22   572   
222  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   228   138   504   
238  /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   414   224  1092   
2    /content/drive/MyDrive/PhatHienDuongLuoiBo/Fas...   476   613   683   

     ymax dash  
229   363  yes  
40    481  yes  
85    

In [7]:
annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/images/*.jpg')

for file in annotations:
  if (check(file,data_test) == False):
    data = [file] + [0,0,0,0] + ['bg']
    df.append(data)

  data_test = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])


In [8]:
data_test['dash'].value_counts()

bg     413
yes    244
Name: dash, dtype: int64

In [9]:
data_test.to_csv('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/test/annotations.txt',index=False,header=False)

> # **1. processdata**

In [ ]:
from IPython.core.display import Image
import cv2
import os
import random
import shutil 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np
from google.colab.patches import cv2_imshow
def draw_bounding_box(image, annotation_text_file):
  with open(annotation_text_file, 'r') as file:
    annotation = file.read().split('\n')[:-1]
    annotation = [ x.split(" ") for x in annotation]
    annotation = [ list(map(float,x)) for x in annotation]

    
  annotations = np.array(annotation) #has a from [[class_id, x_center, y_center, bbox_width, bbox_height]]

  print(f"annotation value before de-normalizaiton: {annotations}")
  h , w, _ = image.shape
  print("width:",h)
  print("height:",w)
  # de-normalize annotations
  annotations_cp = np.copy(annotations)
  annotations_cp[:, [1,3]] = annotations[:, [1,3]] * w # x_center, bbox_width
  annotations_cp[:, [2,4]] = annotations[:, [2,4]] * h # y_center, bbox_height
  print(f"annotation value after de-normolization: {annotations_cp}")

  #convert to (xmin, xmax, ymin, ymax) to draw cv2.rectangle
  annotations_cp[ :,1] = (annotations_cp[ :,1] - annotations_cp[ :,3]/2)
  annotations_cp[ :,2] = (annotations_cp[ :,2] - annotations_cp[ :,4]/2)
  annotations_cp[ :,3] = (annotations_cp[ :,1] + annotations_cp[ :,3])
  annotations_cp[ :,4] = (annotations_cp[ :,2] + annotations_cp[ :,4])

  for single_annotation in annotations_cp:
    obj_cls, x0, y0, x1, y1 = single_annotation
    start_point = (int(x0), int(y0))
    end_point = (int(x1), int(y1))
    color=(255,0,0)
    thickness=2
    cv2.rectangle(image, start_point, end_point, color, thickness)
    cv2_imshow(image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [ ]:
def check(img_name,df):
  for name in df['filename']:
    if img_name == name :
    # if img_name == name.split("/")[-1] :
      return True
  return False

In [ ]:
import os, sys
from glob import glob
import pandas as pd
import cv2 
import numpy as np

annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/train/images/lables/*.txt')

df = []
count = 0;
count_train =0;
for file in annotations:
    filename = file.split('/')[-1]
    filenameJPG = filename.split('.')[0] + '.jpg'
    image_file = "/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/train/images/" + filenameJPG
    img = cv2.imread(image_file)
    height, width,_ = img.shape
    row = []
    with open("/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/train/images/lables/" + filename, 'r') as file:
        annotation = file.read().split('\n')[:-1]
        if annotation == [] :
            count +=1;
            continue; 
        
        annotation = [ x.split(" ") for x in annotation]
        annotation = [ list(map(float,x)) for x in annotation]
    # for annotation_s in annotation: 
    annotations = np.array(annotation) #has a from [[class_id, x_center, y_center, bbox_width, bbox_height]]
    # de-normalize annotations
    annotations_cp = np.copy(annotations)
    annotations_cp[:, [1,3]] = annotations[:, [1,3]] * width # x_center, bbox_width
    annotations_cp[:, [2,4]] = annotations[:, [2,4]] * height # y_center, bbox_height


    #convert to (xmin, xmax, ymin, ymax) to draw cv2.rectangle
    annotations_cp[ :,1] = (annotations_cp[ :,1] - annotations_cp[ :,3]/2)
    annotations_cp[ :,2] = (annotations_cp[ :,2] - annotations_cp[ :,4]/2)
    annotations_cp[ :,3] = (annotations_cp[ :,1] + annotations_cp[ :,3])
    annotations_cp[ :,4] = (annotations_cp[ :,2] + annotations_cp[ :,4])
 
    for single_annotation in annotations_cp:
        dash = "yes"
        obj_cls, x0, y0, x1, y1 = single_annotation
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
        row = [image_file,  x0 , y0, x1, y1, dash]
        df.append(row)
        count_train +=1;

data_train = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])
# data_train[['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash']].to_csv('test_labels.txt', index=False,header=False)

In [ ]:
print("img has value", count)
print("img has value", count_train)

data_train = data_train.sample(frac = 1)
data_train.head

read data background

In [ ]:
annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/train/images/*.jpg')

for file in annotations:
  if (check(file,data_train) == False):
    data = [file] + [0,0,0,0] + ['bg']
    df.append(data)

  data_train = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])


In [ ]:
print("img backgound", count)
data_train = data_train.sample(frac = 1)
data_train.head(20)

In [ ]:
data_train['dash'].value_counts()

save bbox and label


In [ ]:
data_train.to_csv('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/train/annotations.txt',index=False,header=False)

get annotation test data <br>
import

In [ ]:
import os, sys
from glob import glob
import pandas as pd
import cv2 
import numpy as np


annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/test/images/lables/*.txt')

df = []
count = 0;
count_train =0;
for file in annotations:
    filename = file.split('/')[-1]
    filenameJPG = filename.split('.')[0] + '.jpg'
    image_file = "/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/test/images/" + filenameJPG
    img = cv2.imread(image_file)
    height, width,_ = img.shape
    row = []
    with open("/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/test/images/lables/" + filename, 'r') as file:
        annotation = file.read().split('\n')[:-1]
        if annotation == [] :
            count +=1;
            continue; 
        
        annotation = [ x.split(" ") for x in annotation]
        annotation = [ list(map(float,x)) for x in annotation]
    # for annotation_s in annotation: 
    annotations = np.array(annotation) #has a from [[class_id, x_center, y_center, bbox_width, bbox_height]]
    # de-normalize annotations
    annotations_cp = np.copy(annotations)
    annotations_cp[:, [1,3]] = annotations[:, [1,3]] * width # x_center, bbox_width
    annotations_cp[:, [2,4]] = annotations[:, [2,4]] * height # y_center, bbox_height


    #convert to (xmin, xmax, ymin, ymax) to draw cv2.rectangle
    annotations_cp[ :,1] = (annotations_cp[ :,1] - annotations_cp[ :,3]/2)
    annotations_cp[ :,2] = (annotations_cp[ :,2] - annotations_cp[ :,4]/2)
    annotations_cp[ :,3] = (annotations_cp[ :,1] + annotations_cp[ :,3])
    annotations_cp[ :,4] = (annotations_cp[ :,2] + annotations_cp[ :,4])
 
    for single_annotation in annotations_cp:
        dash = "yes"
        obj_cls, x0, y0, x1, y1 = single_annotation
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
        row = [image_file,  x0 , y0, x1, y1, dash]
        df.append(row)
        count_train +=1;

data_test = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])

In [ ]:
print("img has value", count)
print("img has value", count_train)

data_test = data_test.sample(frac = 1)
data_test.head

In [ ]:
annotations = glob('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/test/images/*.jpg')

for file in annotations:
  if (check(file,data_test) == False):
    data = [file] + [0,0,0,0] + ['bg']
    df.append(data)

  data_test = pd.DataFrame(df, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'dash'])


In [ ]:
data_test['dash'].value_counts()

In [ ]:
data_test.to_csv('/content/drive/MyDrive/PhatHienDuongLuoiBo/Faster_RCNN/faster/test/annotations.txt',index=False,header=False)